In [1]:
import numpy as np

from sklearn.metrics import roc_auc_score, precision_score, recall_score, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable, gradcheck
from torch.utils.data import TensorDataset, DataLoader

import pandas as pd

from matplotlib import pyplot

import mne
from mne.io import concatenate_raws, read_raw_fif
import mne.viz

import math

from os import walk

In [2]:
# take a look at some of the data
data_file = 'study1_eeg/study1_EEG_P-01_FN_Trial-001.csv'

data_P_09 = pd.read_csv(data_file)
data_P_09.tail()

,-0.698,-0.762,-0.944,-1.122,-1.108,-0.723,0.07,1.125,2.155,2.898,...,20.833,20.093,19.654,19.536,19.697,20.011,20.335,20.588,20.827,21.199
58,-3.356,-3.230,-2.968,-2.780,-2.813,-3.067,-3.422,-3.729,-3.881,-3.842,...,-5.171,-5.669,-6.558,-7.532,-8.214,-8.319,-7.755,-6.610,-5.066,-3.331
59,-0.936,-1.833,-2.833,-3.619,-3.919,-3.637,-2.916,-2.085,-1.501,-1.373,...,-3.416,-2.966,-2.921,-3.143,-3.330,-3.237,-2.777,-1.950,-0.725,0.973
60,1.232,0.121,-0.805,-1.356,-1.463,-1.185,-0.671,-0.093,0.447,0.963,...,-1.272,-2.243,-3.477,-4.764,-5.790,-6.248,-5.933,-4.799,-2.954,-0.630
61,0.585,0.034,-0.144,0.065,0.452,0.721,0.667,0.278,-0.293,-0.810,...,-7.846,-7.245,-6.739,-6.557,-6.794,-7.355,-8.004,-8.513,-8.786,-8.876
62,4.899,5.001,4.836,4.464,3.933,3.295,2.640,2.102,1.814,1.850,...,-2.848,-2.510,-2.391,-2.514,-2.775,-2.982,-2.920,-2.431,-1.453,-0.030


In [3]:
# take some data that was already formatted, from this link: https://neuro.inf.unibe.ch/AlgorithmsNeuroscience/Tutorial_files/DatasetConstruction.html
data_file = 'study1_eeg/epochdata/master'

# Read the EEG epochs:
epochs = mne.read_epochs(data_file + '.fif', verbose='error')
print(epochs)

<EpochsFIF  |   16666 events (all good), 0 - 1.49609 sec, baseline off, ~3.05 GB, data loaded,
 'FN': 4336
 'FP': 4371
 'FU': 4239
 'NN': 1238
 'NP': 1264
 'NU': 1218>


In [4]:
epochs_UN = epochs['FU', 'FN'] # Unpleasant vs. Neutral
epochs_UP = epochs['FU', 'FP'] # Unpleasant vs. Pleasant
epochs_NP = epochs['FN', 'FP'] # Neutral vs. Pleasant

# Dataset with unpleasant and neutral events
print(epochs_UN)
data_UN = epochs_UN.get_data() #we will classify between unpleasant and neutral
labels_UN = epochs_UN.events[:,-1]
print(len(labels_UN))

<EpochsFIF  |   8575 events (all good), 0 - 1.49609 sec, baseline off, ~1.57 GB, data loaded,
 'FN': 4336
 'FU': 4239>
8575


In [5]:
train_data_UN, test_data_UN, labels_train_UN, labels_test_UN = train_test_split(data_UN, labels_UN, test_size=0.3, random_state=42)

In [6]:
print(labels_train_UN.shape, labels_test_UN.shape)

(6002,) (2573,)


In [7]:
BATCH_SIZE = 301

eeg_data_scaler = StandardScaler()

X_train = eeg_data_scaler.fit_transform(train_data_UN.reshape(-1, train_data_UN.shape[-1])).reshape(train_data_UN.shape)
X_test = eeg_data_scaler.fit_transform(test_data_UN.reshape(-1, test_data_UN.shape[-1])).reshape(test_data_UN.shape)

labels_train_UN = np.array([1 if x > 0 else 0 for x in labels_train_UN])
labels_test_UN = np.array([1 if x > 0 else 0 for x in labels_test_UN])

labels_train_UN = labels_train_UN.reshape((labels_train_UN.shape[0], 1))
labels_train_UN = labels_train_UN.astype(np.float32)
X_actual = torch.from_numpy(labels_train_UN)

labels_test_UN = labels_test_UN.reshape((labels_test_UN.shape[0], 1))
labels_test_UN = labels_test_UN.astype(np.float32)
X_test_actual = torch.from_numpy(labels_test_UN)

X_train = torch.from_numpy(X_train)
X_train = X_train.unsqueeze(1)
X_test = torch.from_numpy(X_test)
X_test = X_test.unsqueeze(1)

X_list = [0] * (math.ceil(X_train.shape[0] / BATCH_SIZE))
for i in range(len(X_list)):
    a, b = BATCH_SIZE * i, BATCH_SIZE * (i + 1)
    if i != len(X_list) - 1:
        X_list[i] = (X_train[a:b, :, : ], X_actual[a:b, :])
    else:
        X_list[i] = (X_train[a:, :, : ], X_actual[a:, :])


print(X_train.shape, X_actual.shape, X_test.shape)

torch.Size([6002, 1, 64, 384]) torch.Size([6002, 1]) torch.Size([2573, 1, 64, 384])


In [8]:
conv1_neurons = 4
conv2_neurons = 8
conv3_neurons = 4
flat1_in = 168
flat1_out = 64
flat2_out = 256
flat3_out = 64

In [9]:
CNNPoor = nn.Sequential(
    nn.Conv2d(1, conv1_neurons, (1, 128), padding=1),
    nn.ELU(),
    nn.BatchNorm2d(conv1_neurons, False),
    
    nn.Conv2d(conv1_neurons, conv2_neurons, (64, 1), padding=1),
    nn.ELU(),
    nn.BatchNorm2d(conv2_neurons, False),
    nn.AvgPool2d((1, 4)),
    nn.Dropout(),
    
    nn.Conv2d(conv2_neurons, conv3_neurons, (1, 16), padding=1),
    nn.ELU(),
    nn.BatchNorm2d(conv3_neurons, False),
    nn.AvgPool2d((1, 8)),
    nn.Dropout(),
    
    nn.Flatten(),

    nn.Linear(flat1_in, flat1_out),
    nn.ELU(),
    nn.Linear(flat1_out, 1),
    nn.Sigmoid(),
)

In [10]:
loss_function = nn.BCELoss()
optimizer = optim.Adam(CNNPoor.parameters(), lr = 0.001)

In [11]:
def evaluate(model, data):
    
    with torch.no_grad():
        content, labels = data
        pred = model(content)
        pred = pred.numpy()
    return accuracy_score(labels, np.round(pred))

In [13]:
for i in range(25):
    print("Epoch: ", i)
    tot_loss = 0.0

    for j in range(math.ceil(X_train.shape[0] / BATCH_SIZE)):
        data, labels = X_list[j]
        data, labels = Variable(data.float()), Variable(labels)
        
        optimizer.zero_grad()
        
        classification = CNNPoor(data)
        loss = loss_function(classification, labels)
        loss.backward()
        
        optimizer.step()

        tot_loss += loss.item()
    print("Total loss = ", tot_loss)
    print("Train accuracy = ", evaluate(CNNPoor, (X_train.float(), X_actual)))
    print("Test accuracy = ", evaluate(CNNPoor, (X_test.float(), X_test_actual)))

Epoch:  0
Total loss =  12.569632589817047
Train accuracy =  0.653615461512829
Test accuracy =  0.5787019043917606
Epoch:  1
Total loss =  12.575608730316162
Train accuracy =  0.64595134955015
Test accuracy =  0.5616012436844151
Epoch:  2
Total loss =  12.529138326644897
Train accuracy =  0.6607797400866378
Test accuracy =  0.5744267392149243
Epoch:  3
Total loss =  12.556964874267578
Train accuracy =  0.6514495168277241
Test accuracy =  0.5779246016323358
Epoch:  4
Total loss =  12.50970470905304
Train accuracy =  0.6584471842719094
Test accuracy =  0.5806451612903226
Epoch:  5
Total loss =  12.43011748790741
Train accuracy =  0.6591136287904032
Test accuracy =  0.586863583365721
Epoch:  6
Total loss =  12.418574512004852
Train accuracy =  0.6607797400866378
Test accuracy =  0.567431014380101
Epoch:  7
Total loss =  12.402537107467651
Train accuracy =  0.6649450183272243
Test accuracy =  0.5643218033424019
Epoch:  8
Total loss =  12.242958128452301
Train accuracy =  0.6644451849383539